## Local critical thresholds.

In [4]:
from graph_reciprocity import *

In [15]:
#Speed Test
N_nodes = 100
p_edge = 0.5
p = 0.5
num_trials = 1
p_coop = 1.0
#get_average_mutual_neighbors_for_graph_type(N_nodes,p_edge,p,num_trials,create_graph,random_number=False)
G = create_graph(N_nodes,p_edge,p_coop,random_number=False)
%time get_average_p_known(G,prob_of_known_fn = lambda x,y: get_p_known(x,y,p))

CPU times: user 272 ms, sys: 9.75 ms, total: 281 ms
Wall time: 369 ms


(0.9992920416972262, 25.301751592356688)

In [24]:
import networkx as nx
G_fb = nx.read_edgelist('../data/facebook_combined.txt',nodetype=int)

In [130]:
G = nx.ego_graph(G_fb,random.choice(G_fb.nodes(),size=1,replace=False)[0])
print get_degree_and_average_threshold(G,p)
degrees = nx.degree(G).values()
bins = logspace(log10(min(degrees)),log10(max(degrees)),num=40)
hist(nx.degree(G).values(),bins=bins,normed=True,log=True)
gca().set_xscale('log')
grid()

(25.3125, 1.494874496832746)


In [151]:
N_range = array([1000])#array([0.05,0.2,0.6])
#close('all')
p = 0.2
newdata = False
for j,N_nodes in enumerate(N_range):
    num_trials_default = 1
    p_coop = 1 - 1.0/N_nodes
    p_edge_range = array([0.1,0.25,0.5,1,2,4])*sqrt(5.0/N_nodes)#array([0.1,0.25,0.5,1,2,3,4])*sqrt(5.0/N_nodes)
    num_trials_array = num_trials_default*ones_like(p_edge_range)
    # num_trials_array[0:3] *= 5
    # num_trials_array[0:1] *= 5
    p_know_range_th = get_p_known_first_order(p_edge_range**2*N_nodes,p)
    p_know_range = zeros_like(p_edge_range)
    p_know_range_sw = zeros_like(p_edge_range)
    p_know_range_sf = zeros_like(p_edge_range)
    mutual_neighbors_range = zeros_like(p_edge_range)
    mutual_neighbors_range_sw = zeros_like(p_edge_range)
    mutual_neighbors_range_sf = zeros_like(p_edge_range)
    num_edges_range = zeros_like(p_edge_range)
    num_edges_range_sw = zeros_like(p_edge_range)
    num_edges_range_sf = zeros_like(p_edge_range)



    for i,p_edge in enumerate(p_edge_range):
        num_trials = int(round(num_trials_array[i]))
        p_know_range[i],mutual_neighbors_range[i],num_edges_range[i] = get_average_mutual_neighbors_for_graph_type(N_nodes,p_edge,p,num_trials,create_graph,random_number=False)
        p_know_range_sw[i],mutual_neighbors_range_sw[i],num_edges_range_sw[i] = get_average_mutual_neighbors_for_graph_type(N_nodes,p_edge,p,num_trials,create_small_world_graph,random_number=False)
        p_know_range_sf[i],mutual_neighbors_range_sf[i],num_edges_range_sf[i] = get_average_mutual_neighbors_for_graph_type(N_nodes,p_edge,p,num_trials,create_scale_free_graph,random_number=False)

    k_range_th = N_nodes*p_edge_range
    
    #plotting
    figure(1)
    k_range = 2.0*num_edges_range/N_nodes
    k_range_sw = 2.0*num_edges_range_sw/N_nodes
    k_range_sf = 2.0*num_edges_range_sf/N_nodes

    axhline(1/p,color = 'k',label=r'Th. $\left(b/c\right)^*$ (zeroth order)')

    plot(k_range_th**2/N_nodes,1/p_know_range_th,'--k',label=r'Th. $\left(b/c\right)_{critical}$ (first order)')

    plot(k_range**2/N_nodes,1/p_know_range,'-ob',label=r'Sim. $\left(b/c\right)_{critical}$ (first order, Erdos-Renyi)')
    plot(k_range_sw**2/N_nodes,1/p_know_range_sw,'-^r',label=r'Sim. $\left(b/c\right)_{critical}$ (first order, Small World)')
    plot(k_range_sf**2/N_nodes,1/p_know_range_sf,'-sg',label=r'Sim. $\left(b/c\right)_{critical}$ (first order, Clustered Scale Free)')

    xlabel(r'$k = N_{nodes} p_{edge}$',size=20)
    ylabel(r'$\left(\frac{b}{c}\right)^*$',size=20)
    grid()
    #axhline(N_nodes*sqrt(1.0/N_nodes*(log(1-1.0*c/b) - log(1- p))/log(1-p**2)))
    if j == len(p_trial_range)-1:
        legend(loc = (1.02,0.63))
    title(r'$p = ' + str(p) + '$, $N_{nodes} = ' + str(N_nodes) + '$',size=20)
    ylim(1,1.1/p)
    #savefig('b_c_critical_vs_k_p' +  str(p) + '.png',bbox_inches='tight')

1000 3499
1000 3000
1000 2990
1000 8814
1000 8000
1000 7933
1000 17771
1000 17000
1000 16710
1000 35562
1000 35000
1000 33774
1000 70555
1000 70000
1000 65090
1000 141608
1000 141000
1000 121118


In [164]:
#sample degree and thresholds from a larger graph through subgraphs

num_samples =5
points = []
for i in range(num_samples):
    #G = nx.ego_graph(G_fb,random.choice(G_fb.nodes(),size=1,replace=False)[0],radius=1,center=True)
    G = G_fb.subgraph(random.choice(G_fb.nodes(),size=random.randint(low=1000,high=2000),replace=False))
    G = list(nx.connected_component_subgraphs(G))[0]
    p = 0.2
    points.append(get_degree_and_average_threshold(G,p))
    print len(G.nodes()),len(G.edges())

pointsx,pointsy = tuple(zip(*points))
figure(3)
scatter(pointsx,pointsy,c='r')

1654 19960
471 2937
1562 14152
1776 19441
1374 16358


In [2]:
def get_degree_and_average_threshold(G,p):
    k = 2.0*len(G.edges())/len(G.nodes())
    p_known,_ = get_average_p_known(G,prob_of_known_fn = lambda x,y: get_p_known(x,y,p))
    return k**2/len(G.nodes()),1.0/p_known

In [3]:
#%pylab qt
p = 0.4

def draw_graph_thresholds(G):
    close('all')
    k = 2*len(G.edges())/len(G.nodes())
    node_color = [G.node[i]['threshold'] for i in G.nodes()]
    node_size = [100.0*len(G.edges([i]))/k for i in G.nodes()]

    subplot(121)
    
    ax1 = hist(node_color,alpha = 0.6,bins=10)
    xlabel(r'$\frac{p}{P_i}$',size=25)
    ylabel(r'frequency',size=15)
    
    subplot(122)
    vmin = min(node_color)
    vmax = max(node_color)
    pos = nx.spring_layout(G)
    node_labels = {i:'{:.2f}'.format(G.node[i]['threshold']) for i in G.nodes()}
    
    ax2 = nx.draw_networkx_nodes(G,pos,alpha=0.6,node_color=node_color,node_size=node_size,labels=node_labels,with_labels=False)
    
    nx.draw_networkx_edges(G,pos,alpha=0.2)
    
    cbar = colorbar(ax2)
    cbar.ax.set_ylabel(r'$\frac{p}{P_i}$',size=25)
    axis('off')
    cbar.ax.set_xlabel(r'node size $\sim k_i$',size=20)
    suptitle(r'$p = ' + str(p) + '$, $N_{nodes} = ' + str(N_nodes) + '$',size=20)

    
    
    
N_nodes = 150
p_edge = sqrt(1.0/N_nodes)
p_coop = 1 - 1.0/N_nodes
#G = nx.read_edgelist('../data/facebook_combined.txt',nodetype=int)
#G = create_small_world_graph(N_nodes,p_edge,p_coop,random_number=False)
update_threshold(G,prob_of_known_fn = lambda x,y: get_p_known(x,y,p))
draw_graph_thresholds(G)



NameError: name 'G' is not defined

In [5]:
%pylab qt
p = 0.4
N_nodes = 100 #1000
p_edge = sqrt(1.0/N_nodes)
p_coop = 1 - 1.0/N_nodes
close('all')
graph_fns = [create_graph,create_small_world_graph,create_scale_free_graph]
graph_names = ['Erdos-Renyi','Small World', 'Clustered Scale Free','Facebook']
all_thresholds = [[],[],[]]
num_trials = 1
for k in range(num_trials):
    for j,fn in enumerate(graph_fns):
        G = fn(N_nodes,p_edge,p_coop,random_number=False)
        if k == 0:
            print len(G.nodes()),len(G.edges())
        update_threshold(G,prob_of_known_fn = lambda x,y: get_p_known(x,y,p))
        all_thresholds[j] += [G.node[i]['threshold'] for i in range(len(G.nodes()))]
#    thresholds = [100.0*len(G.edges([i]))/k for i in range(len(G.nodes()))]




    
plot_histograms(graph_fns,all_thresholds,p)

N_nodes = 100
p_edge = sqrt(1.0/N_nodes)

all_graphs = []
all_thresholds = []
all_degrees = []
for j,fn in enumerate(graph_fns):
    G = fn(N_nodes,p_edge,p_coop,random_number=False)
    all_graphs.append(G)
    print len(G.nodes()),len(G.edges())
    update_threshold(G,prob_of_known_fn = lambda x,y: get_p_known(x,y,p))

    thresholds = [G.node[i]['threshold'] for i in range(len(G.nodes()))]
    degrees = [len(G.edges([i])) for i in range(len(G.nodes()))]
    
    all_thresholds.append(thresholds)
    all_degrees.append(degrees)    


    
plot_colored_graphs(graph_fns,all_graphs,all_thresholds,all_degrees,graph_names,p)

Populating the interactive namespace from numpy and matplotlib
100 529
100 500
100 475
100 494
100 500
100 474
1.25131801247 2.5
1.25131801247 2.5
1.25131801247 2.5


`%matplotlib` prevents importing * from pylab and numpy


- How to plot graph data? Vs k or versus n?
- How to properly sample from the FB graph?
- Add functionality for 2nd degree and wider paths. Analyze the nodes with the highest incentive to cooperate.